In [1]:
import pandas as pd

In [3]:
articles = pd.read_parquet("data/ebnerd_large/articles.parquet")
train_hist = pd.read_parquet("data/ebnerd_large/train/history.parquet")
train_behaviors = pd.read_parquet("data/ebnerd_large/train/behaviors.parquet")
# val_hist = pd.read_parquet("data/ebnerd_large/validation/history.parquet")
# val_behaviors = pd.read_parquet("data/ebnerd_large/validation/behaviors.parquet")

In [9]:
articles.head()#['subcategory'].nunique()

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3000022,Hanks beskyldt for mishandling,Tom Hanks har angiveligt mishandlet sin afdøde...,2023-06-29 06:20:32,False,Tom Hanks skulle angiveligt have mishandlet si...,2006-09-20 09:24:18,[3518381],article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,[PER],"[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.9911,Negative
1,3000063,Bostrups aske spredt i Furesøen,Studieværten blev mindet med glad festlighed,2023-06-29 06:20:32,False,Strålende sensommersol. Jazzede toner. Glas me...,2006-09-24 07:45:30,"[3170935, 3170939]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kendt, Underholdning, Begivenhed, Personlig b...",118,[133],nyheder,NaN,NaN,NaN,0.5155,Neutral
2,3000613,Jesper Olsen ramt af hjerneblødning,Den tidligere danske landsholdsspiller i fodbo...,2023-06-29 06:20:33,False,"Jesper Olsen, der er noteret for 43 kampe på d...",2006-05-09 11:29:00,[3164998],article_default,https://ekstrabladet.dk/sport/fodbold/landshol...,...,"[LOC, PER, PER, PER]","[Kendt, Sport, Fodbold, Sundhed, Sygdom og beh...",142,"[196, 271]",sport,NaN,NaN,NaN,0.9876,Negative
3,3000700,Madonna topløs med heste,47-årige Madonna poserer både topløs og sammen...,2023-06-29 06:20:33,False,Skal du have stillet Madonna-sulten inden konc...,2006-05-04 11:03:12,[3172046],article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,[],"[Kendt, Livsstil, Underholdning]",414,[432],underholdning,NaN,NaN,NaN,0.8786,Neutral
4,3000840,Otto Brandenburg er død,Sangeren og skuespilleren Otto Brandenburg er ...,2023-06-29 06:20:33,False,"'Og lidt for Susanne, Birgitte og Hanne... ' '...",2007-03-01 18:34:00,[3914446],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kendt, Underholdning, Begivenhed, Personlig b...",118,[133],nyheder,NaN,NaN,NaN,0.9468,Negative


In [10]:
train_behaviors.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,47727,NaN,2023-05-21 21:35:07,20.0,NaN,1,"[9482380, 9775183, 9744403, 9775297, 9774020, ...",[9775183],18293,False,NaN,NaN,NaN,False,265,34.0,100.0
1,47731,NaN,2023-05-21 21:32:33,13.0,NaN,1,"[9774557, 9774516, 9775331, 9775277, 9759966]",[9759966],18293,False,NaN,NaN,NaN,False,265,45.0,100.0
2,47736,NaN,2023-05-21 21:33:32,17.0,NaN,1,"[9759966, 9774557, 9775352, 9746360, 9772601, ...",[9774652],18293,False,NaN,NaN,NaN,False,265,78.0,100.0
3,47737,NaN,2023-05-21 21:38:17,27.0,NaN,1,"[9774580, 9775131, 9775202, 9774789, 9774972, ...",[9775184],18293,False,NaN,NaN,NaN,False,265,6.0,52.0
4,47740,NaN,2023-05-21 21:36:02,48.0,NaN,1,"[9774826, 9775171, 9775076, 9769624, 9775056, ...",[9774648],18293,False,NaN,NaN,NaN,False,265,32.0,100.0


In [13]:
len(train_behaviors)

12063890

In [11]:
train_hist.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,10029,"[2023-04-28T06:16:57.000000, 2023-04-28T06:17:...","[23.0, 69.0, 27.0, nan, 47.0, 38.0, 100.0, 12....","[9735579, 9739888, 9739471, 9739864, 9738441, ...","[28.0, 24.0, 11.0, 107.0, 8.0, 7.0, 20.0, 5.0,..."
1,10033,"[2023-04-27T11:11:32.000000, 2023-04-27T11:12:...","[33.0, 41.0, 33.0, 100.0, 68.0, 38.0, 1.0, 58....","[9738139, 9738263, 9738139, 9738760, 9738777, ...","[2.0, 2.0, 718.0, 18.0, 26.0, 78.0, 3.0, 11.0,..."
2,10034,"[2023-04-30T09:46:57.000000, 2023-04-30T09:47:...","[nan, 88.0, 27.0, nan, 23.0, 100.0, 100.0, 22....","[9742693, 9742686, 9744016, 9743818, 9744922, ...","[21.0, 103.0, 28.0, 0.0, 5.0, 34.0, 14.0, 14.0..."
3,10041,"[2023-04-27T15:15:28.000000, 2023-04-27T15:16:...","[78.0, 41.0, 4.0, 16.0, 22.0, 32.0, 11.0, 94.0...","[9739035, 9738303, 9737243, 9739634, 9739802, ...","[12.0, 11.0, 3.0, 3.0, 4.0, 13.0, 29.0, 24.0, ..."
4,10103,"[2023-04-27T15:37:35.000000, 2023-04-27T15:38:...","[100.0, nan, 100.0, 100.0, 100.0, 28.0, 82.0, ...","[9739035, 9739164, 9741803, 9740087, 9741986, ...","[45.0, 8.0, 61.0, 72.0, 56.0, 3.0, 22.0, 16.0,..."


In [12]:
train_hist['user_id'].nunique()

788090

### validation

In [16]:
# val_hist = pd.read_parquet("data/ebnerd_large/validation/history.parquet")
val_behaviors = pd.read_parquet("data/ebnerd_large/validation/behaviors.parquet")

In [17]:
len(val_behaviors)

12566385

### test

In [14]:
# test_hist = pd.read_parquet("data/ebnerd_large/test/history.parquet")
test_behaviors = pd.read_parquet("data/ebnerd_large/test/behaviors.parquet")

In [15]:
len(test_behaviors)

13536710